## Análise de Fundos Imobiliários 
    Thiago Laidler Vidal Cunha

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import time
import multiprocessing as mp
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import poisson
from scipy.optimize import curve_fit

In [2]:
#Dados retirados em 4 de Julho de 2022
df = pd.read_csv('statusinvest-busca-avancada.csv',
                 #error_bad_lines=False
                sep=';', comment='#', na_values=' ')

In [3]:
df.DY = df.DY.str.replace(".","").str.replace(",",".").astype(float)
df['P/VP'] = df['P/VP'].str.replace(".","").str.replace(",",".").astype(float)
df['PATRIMONIO'] = df['PATRIMONIO'].str.replace(".","").str.replace(",",".").astype(float)
df['ULTIMO DIVIDENDO'] = df['ULTIMO DIVIDENDO'].str.replace(".","").str.replace(",",".").astype(float)
df['PRECO'] = df['PRECO'].str.replace(".","").str.replace(",",".").astype(float)

In [4]:
df

,TICKER,PRECO,ULTIMO DIVIDENDO,DY,VALOR PATRIMONIAL COTA,P/VP,LIQUIDEZ MEDIA DIARIA,PERCENTUAL EM CAIXA,CAGR DIVIDENDOS 3 ANOS,CAGR VALOR CORA 3 ANOS,PATRIMONIO,N COTISTAS,GESTAO,N COTAS
0,VIUR11,7.05,0.072000,11.87,"9,51",0.74,"191.982,87","11,56",NaN,NaN,2.562974e+08,"4.432,00",Ativa,"26.946.220,00"
1,SADI11,90.62,1.060000,9.70,"100,97",0.90,"650.705,70","2,62",NaN,NaN,3.210977e+08,"3.603,00",Ativa,"3.180.207,00"
2,THRA11,138.50,0.130000,32.01,"9,54",14.52,"53.642,50","140,47","90,73","0,12",1.405068e+07,"2.084,00",Passiva,"1.472.728,00"
3,BMII11,0.00,0.228341,0.00,"495,28",NaN,NaN,"0,71",NaN,NaN,6.967589e+07,"12,00",Passiva,"140.680,00"
4,CYCR11,101.94,1.150000,12.40,"97,76",1.04,"43.111,43","30,15",NaN,NaN,7.234444e+07,"377,00",Ativa,"740.046,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,NEWU11,31.58,0.110000,0.00,"64,86",0.49,"8.194,26","1,82","58,77",NaN,4.812746e+07,"1.786,00",Ativa,"742.000,00"
409,FAMB11B,790.00,9.160000,0.00,"3.082,39",0.26,"14.907,91","3,47","-9,82","-37,20",3.230342e+08,"2.960,00",Passiva,"104.800,00"
410,VCRR11,70.94,0.670000,11.33,"119,79",0.59,"104.585,35","0,83",NaN,NaN,2.395757e+08,"2.115,00",Ativa,"2.000.000,00"
411,HSLG11,82.70,0.650000,8.82,"110,72",0.75,"549.540,52","3,78",NaN,NaN,1.401753e+09,"19.657,00",Ativa,"12.660.067,00"


In [5]:
Filtro_DY = df[(df.DY >= 6.0)*(df.DY < 10.0)] # Aqueles com DY entre 6 e 9.9
Filtro = Filtro_DY[Filtro_DY['P/VP'] <= 1.0] #Aqueles com valor de cota menor ou igual ao valor patrimonial
Filtro = Filtro.sort_values(by=['PATRIMONIO'], ascending=False) #ascending=False para organizar de ordem decrescente
Filtro #Organizando pelo valor do patrimonio

/home/laidler/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:177: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


,TICKER,PRECO,ULTIMO DIVIDENDO,DY,VALOR PATRIMONIAL COTA,P/VP,LIQUIDEZ MEDIA DIARIA,PERCENTUAL EM CAIXA,CAGR DIVIDENDOS 3 ANOS,CAGR VALOR CORA 3 ANOS,PATRIMONIO,N COTISTAS,GESTAO,N COTAS
370,PBLV11,926.98,36.280000,6.76,"1.597,84",0.58,NaN,"1,71",NaN,NaN,5.137604e+09,"2,00",Ativa,"3.215.347,00"
356,KNCR11,100.74,1.050000,9.20,"101,13",1.00,"7.458.565,74","18,86","19,05","-0,70",4.036598e+09,"112.554,00",Ativa,"39.916.179,00"
29,KNRI11,131.00,0.840000,7.30,"157,13",0.83,"2.735.828,00","2,64","3,91","-5,04",3.795886e+09,"234.264,00",Ativa,"24.157.121,00"
341,XPLG11,94.29,0.700000,8.27,"113,75",0.83,"4.252.505,83","1,31","2,33","-2,80",3.082675e+09,"281.409,00",Ativa,"27.101.061,00"
170,BRCR11,60.05,0.470000,9.59,"101,51",0.59,"1.747.931,30","1,06","-0,60","-14,98",2.703973e+09,"140.193,00",Ativa,"26.638.202,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,BICR11,99.01,0.610000,7.52,"100,17",0.99,"3.998,11","28,67",NaN,NaN,5.008508e+07,"554,00",Ativa,"500.000,00"
201,IBFF11,61.73,0.500000,9.85,"72,28",0.85,"120.969,43","5,28",NaN,NaN,4.819082e+07,"4.265,00",Ativa,"666.681,00"
368,LSPA11,107.30,0.750000,7.86,"107,38",1.00,"2.294.589,00","0,81",NaN,NaN,4.343672e+07,"223,00",Ativa,"404.500,00"
28,REIT11,480.00,2.063629,7.34,"486,27",0.99,"960,00","41,23",NaN,NaN,1.278465e+07,"62,00",Ativa,"26.291,00"


    A ideia agora é pegar os 10 primeiros fundos, pegar os valor de cota (coluna preço) e do ultimo dividendo.
    Criar função que calcule o valor necessário para comprar 1 cota com o valor dos proprios dividendos.
    Os melhores fundos serão, então, analisados mais a fundo no google. Aqueles com gráfico de dividendos mais estáveis e com menor vacância serão os ideais.

 - Dando nome para cada linha a fim de facilitar a feitura da função. Aproveitei para renomear algumas colunas também.

In [6]:
df_2 = pd.DataFrame(data = Filtro[1:11].values,index = list(range(1,11)),columns = ['Sigla','Preco','Ultimo Dividendo','DY','VALOR PATRIMONIAL COTA','P/VP','LIQUIDEZ MEDIA DIARIA','PERCENTUAL EM CAIXA','CAGR DIVIDENDOS 3 ANOS','CAGR VALOR CORA 3 ANOS','PATRIMONIO','COTISTAS','GESTAO','N COTAS'])
df_2

,Sigla,Preco,Ultimo Dividendo,DY,VALOR PATRIMONIAL COTA,P/VP,LIQUIDEZ MEDIA DIARIA,PERCENTUAL EM CAIXA,CAGR DIVIDENDOS 3 ANOS,CAGR VALOR CORA 3 ANOS,PATRIMONIO,COTISTAS,GESTAO,N COTAS
1,KNCR11,100.74,1.05,9.2,"101,13",1,"7.458.565,74","18,86","19,05","-0,70",4.0366e+09,"112.554,00",Ativa,"39.916.179,00"
2,KNRI11,131,0.84,7.3,"157,13",0.83,"2.735.828,00","2,64","3,91","-5,04",3.79589e+09,"234.264,00",Ativa,"24.157.121,00"
3,XPLG11,94.29,0.7,8.27,"113,75",0.83,"4.252.505,83","1,31","2,33","-2,80",3.08268e+09,"281.409,00",Ativa,"27.101.061,00"
4,BRCR11,60.05,0.47,9.59,"101,51",0.59,"1.747.931,30","1,06","-0,60","-14,98",2.70397e+09,"140.193,00",Ativa,"26.638.202,00"
5,JSRE11,77.61,0.49,8,"113,63",0.68,"3.474.669,48","0,11","1,04","-11,05",2.35989e+09,"56.493,00",Ativa,"20.767.328,00"
6,HGRU11,117.8,1.2,8.1,"119,05",0.99,"4.055.242,00","4,72","5,09","2,01",2.19121e+09,"165.214,00",Ativa,"18.406.458,00"
7,HGBS11,170.2,1.4,7.4,"216,52",0.79,"1.202.744,04","0,83","-1,41","-12,89",2.16519e+09,"80.779,00",Ativa,"10.000.000,00"
8,VISC11,99.18,0.7,7.84,"115,15",0.87,"2.050.288,52","1,95","4,88","-2,54",2.03776e+09,"208.938,00",Ativa,"17.696.234,00"
9,BCFF11,63.89,0.54,9.65,"78,11",0.82,"2.415.270,91","5,64","-0,25","-10,94",1.9673e+09,"296.075,00",Ativa,"25.186.378,00"
10,HGRE11,131.5,0.78,8.07,"162,86",0.81,"1.768.950,35","0,78","7,17","-6,62",1.9246e+09,"125.469,00",Ativa,"11.817.767,00"


In [7]:
def BolaDeNeve(data):
    '''Docstring:
        Função recebe Data Frame e retorna valores para calcularmos o Magic Number necessário para
        começar a 'bola de neve'. Ou seja, a quantidade de cotas e a quantidade de investimento necessario
        para que a rentabilidade pague +1 cota.    
    '''
    cotas=[]
    valores=[]
    for i in range(1,len(data)+1):
        preco = data.loc[i]['Preco']
        div_ult = data.loc[i]['Ultimo Dividendo']
        cotas_nec = round(preco/div_ult)
        valor_nec = cotas_nec*preco
        cotas.append(cotas_nec)
        valores.append(valor_nec)
    return cotas,valores

In [8]:
#Magic Number
df_2['Cotas Necessarias'],df_2['Investimento Necessario (Real)'] = BolaDeNeve(df_2)

 - Ordenando com base no menor valor necessário para alcançar o Magic Number:

In [9]:
df_2.sort_values(by=['Investimento Necessario (Real)'])

,Sigla,Preco,Ultimo Dividendo,DY,VALOR PATRIMONIAL COTA,P/VP,LIQUIDEZ MEDIA DIARIA,PERCENTUAL EM CAIXA,CAGR DIVIDENDOS 3 ANOS,CAGR VALOR CORA 3 ANOS,PATRIMONIO,COTISTAS,GESTAO,N COTAS,Cotas Necessarias,Investimento Necessario (Real)
9,BCFF11,63.89,0.54,9.65,"78,11",0.82,"2.415.270,91","5,64","-0,25","-10,94",1.9673e+09,"296.075,00",Ativa,"25.186.378,00",118,7539.02
4,BRCR11,60.05,0.47,9.59,"101,51",0.59,"1.747.931,30","1,06","-0,60","-14,98",2.70397e+09,"140.193,00",Ativa,"26.638.202,00",128,7686.40
1,KNCR11,100.74,1.05,9.2,"101,13",1,"7.458.565,74","18,86","19,05","-0,70",4.0366e+09,"112.554,00",Ativa,"39.916.179,00",96,9671.04
6,HGRU11,117.8,1.2,8.1,"119,05",0.99,"4.055.242,00","4,72","5,09","2,01",2.19121e+09,"165.214,00",Ativa,"18.406.458,00",98,11544.40
5,JSRE11,77.61,0.49,8,"113,63",0.68,"3.474.669,48","0,11","1,04","-11,05",2.35989e+09,"56.493,00",Ativa,"20.767.328,00",158,12262.38
3,XPLG11,94.29,0.7,8.27,"113,75",0.83,"4.252.505,83","1,31","2,33","-2,80",3.08268e+09,"281.409,00",Ativa,"27.101.061,00",135,12729.15
8,VISC11,99.18,0.7,7.84,"115,15",0.87,"2.050.288,52","1,95","4,88","-2,54",2.03776e+09,"208.938,00",Ativa,"17.696.234,00",142,14083.56
2,KNRI11,131,0.84,7.3,"157,13",0.83,"2.735.828,00","2,64","3,91","-5,04",3.79589e+09,"234.264,00",Ativa,"24.157.121,00",156,20436.00
7,HGBS11,170.2,1.4,7.4,"216,52",0.79,"1.202.744,04","0,83","-1,41","-12,89",2.16519e+09,"80.779,00",Ativa,"10.000.000,00",122,20764.40
10,HGRE11,131.5,0.78,8.07,"162,86",0.81,"1.768.950,35","0,78","7,17","-6,62",1.9246e+09,"125.469,00",Ativa,"11.817.767,00",169,22223.50


 - Ordenando com base na maior rentabilidade:

In [10]:
# Para facilitar a comparação com a renda fixa:
df_2['% ao mês'] = (df_2['Preco']/df_2['Investimento Necessario (Real)'])*100
df_2.sort_values(by=['% ao mês'], ascending=False)

,Sigla,Preco,Ultimo Dividendo,DY,VALOR PATRIMONIAL COTA,P/VP,LIQUIDEZ MEDIA DIARIA,PERCENTUAL EM CAIXA,CAGR DIVIDENDOS 3 ANOS,CAGR VALOR CORA 3 ANOS,PATRIMONIO,COTISTAS,GESTAO,N COTAS,Cotas Necessarias,Investimento Necessario (Real),% ao mês
1,KNCR11,100.74,1.05,9.2,"101,13",1,"7.458.565,74","18,86","19,05","-0,70",4.0366e+09,"112.554,00",Ativa,"39.916.179,00",96,9671.04,1.04167
6,HGRU11,117.8,1.2,8.1,"119,05",0.99,"4.055.242,00","4,72","5,09","2,01",2.19121e+09,"165.214,00",Ativa,"18.406.458,00",98,11544.40,1.02041
9,BCFF11,63.89,0.54,9.65,"78,11",0.82,"2.415.270,91","5,64","-0,25","-10,94",1.9673e+09,"296.075,00",Ativa,"25.186.378,00",118,7539.02,0.847458
7,HGBS11,170.2,1.4,7.4,"216,52",0.79,"1.202.744,04","0,83","-1,41","-12,89",2.16519e+09,"80.779,00",Ativa,"10.000.000,00",122,20764.40,0.819672
4,BRCR11,60.05,0.47,9.59,"101,51",0.59,"1.747.931,30","1,06","-0,60","-14,98",2.70397e+09,"140.193,00",Ativa,"26.638.202,00",128,7686.40,0.78125
3,XPLG11,94.29,0.7,8.27,"113,75",0.83,"4.252.505,83","1,31","2,33","-2,80",3.08268e+09,"281.409,00",Ativa,"27.101.061,00",135,12729.15,0.740741
8,VISC11,99.18,0.7,7.84,"115,15",0.87,"2.050.288,52","1,95","4,88","-2,54",2.03776e+09,"208.938,00",Ativa,"17.696.234,00",142,14083.56,0.704225
2,KNRI11,131,0.84,7.3,"157,13",0.83,"2.735.828,00","2,64","3,91","-5,04",3.79589e+09,"234.264,00",Ativa,"24.157.121,00",156,20436.00,0.641026
5,JSRE11,77.61,0.49,8,"113,63",0.68,"3.474.669,48","0,11","1,04","-11,05",2.35989e+09,"56.493,00",Ativa,"20.767.328,00",158,12262.38,0.632911
10,HGRE11,131.5,0.78,8.07,"162,86",0.81,"1.768.950,35","0,78","7,17","-6,62",1.9246e+09,"125.469,00",Ativa,"11.817.767,00",169,22223.50,0.591716


 - Percentual em caixa muito alto significa que boa parte do dinheiro do acionista não está sendo usado de forma correta, mas um valor baixo demais também não é legal pois pode causar problemas pela falta de liquidez
 - É importante analisar a vacância do fundo (foco na Física e Financeira), essa info pode ser obtida pelo site fundesexplorer.
 - A data de pagamento dos dividendos pode ser vista no statusinvest.
 - É legal refazer essa análise com as tabelas .csv de 4 setores diferentes a fim de diversificar a carteira (esta primeira análise considerou todos os setores de uma vez).